In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import losses_utils
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
IMAGE_SIZE = [224, 224]
train_path = 'C:/Users/lenovo/Desktop/deep_learning_face_recogn/Train'
valid_path = 'C:/Users/lenovo/Desktop/deep_learning_face_recogn/Test'

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
  

folders = glob(train_path)
x = Flatten()(vgg.output)
prediction = Dense(4, activation='relu')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss= 'categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_generator = train_datagen.flow_from_directory('C:/Users/lenovo/Desktop/deep_learning_face_recogn/Train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = "categorical")

test_generator = test_datagen.flow_from_directory('C:/Users/lenovo/Desktop/deep_learning_face_recogn/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = None)

r = model.fit_generator(
 generator=valid_generator,
                    steps_per_epoch=len(valid_generator),
                    validation_data=valid_generator,
                    validation_steps=len(test_generator),
                    epochs=3
)

model.evaluate_generator(generator=valid_generator,
steps=len(test_generator))
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

model.save('C:/Users/lenovo/Desktop/deep_learning_face_recogn/mymodel.h5')


Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

KeyboardInterrupt: 

In [ ]:
from PIL import Image
from keras.applications.vgg19 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
model = load_model('C:/Users/lenovo/Desktop/mask_face/maskmodel.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces=face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)                   
        name="None matching"
        
        if(pred[0][0]):
            name='Abhishek'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        elif(pred[0][0] ):
            name='Mom'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (139,0,0), 2)          
        elif(pred[0][0]):
            name='Abha'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,0), 2)               
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(2)==27:
                break    
video_capture.release()
cv2.destroyAllWindows()


